In [1]:
import os, shutil

RAW_DIR = "../data/raw/images/FaceForensics++_C23"
PROC_DIR = "../data/processed"

real_dir = os.path.join(RAW_DIR, "original")
fake_dirs = ["Deepfakes", "Face2Face", "FaceShifter", "FaceSwap", "NeuralTextures", "DeepFakeDetection"]

# Make target dirs
os.makedirs(os.path.join(PROC_DIR, "real"), exist_ok=True)
os.makedirs(os.path.join(PROC_DIR, "fake"), exist_ok=True)

# Copy REAL images
for file in os.listdir(real_dir):
    src = os.path.join(real_dir, file)
    dst = os.path.join(PROC_DIR, "real", file)
    shutil.copy(src, dst)

# Copy FAKE images
for folder in fake_dirs:
    fpath = os.path.join(RAW_DIR, folder)
    for file in os.listdir(fpath):
        src = os.path.join(fpath, file)
        dst = os.path.join(PROC_DIR, "fake", f"{folder}_{file}")  # prefix filename
        shutil.copy(src, dst)

print("Real & Fake data grouped into processed/real and processed/fake")


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../data/raw/images/FaceForensics++_C23\\original'